In [1]:
import numpy as np
import keras
import random

Using TensorFlow backend.


In [11]:
# generate random data

f = open("data.csv", 'w')

for _ in range(100000):
    x1 = random.random()
    x2 = random.random()
    y = np.sin(x1) + np.exp(x2) + 0.1 *random.random()
    print(','.join([str(x1), str(x2), str(y)]), file=f)
    

f = open("data_test.csv", 'w')

for _ in range(100000):
    x1 = random.random()
    x2 = random.random()
    y = np.sin(x1)  + np.exp(x2) + 0.1 * random.random()
    print(','.join([str(x1), str(x2), str(y)]), file=f)

In [12]:

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, file_cursor, total_num_data, num_features, batch_size=32,shuffle=True):
        """
            file_cursor is an opened file, WITHOUT .read() 
        """
        self.total_num_data = total_num_data
        self.num_features = num_features
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_cursor = file_cursor
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.total_num_data / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'

        X, y = self.__data_generation()

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch, reopen the file'
        self.file_cursor.seek(0)

    def __data_generation(self):
        """read self.batch_size of examples"""
        
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.num_features))
        y = np.empty((self.batch_size,))

        for i in range(self.batch_size):
            line = self.file_cursor.readline()
            # parse string to numpy
            line = [float(v) for v in line[:-1].split(',')]
            X[i,:] = line[:-1]
            y[i] = line[-1]
        
        return X, y

In [13]:
f = open("data.csv", 'r')
gen = DataGenerator(file_cursor=f, total_num_data=100000, num_features=2)

In [14]:
f = open("data.csv", 'r')
for line in f.readlines():
    line = [float(v) for v in line[:-1].split(',')]
    if len(line)!=3:
        print(line)

In [15]:
X, y = gen.__getitem__(0)

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD



# Generators
training_generator = DataGenerator(open("data.csv", 'r'), total_num_data=100000, num_features=2)
validation_generator = DataGenerator(open("data_test.csv", 'r'), total_num_data=100000, num_features=2)

# Design model

# 如果要改模型， 只需要改这个东西

model = Sequential()
model.add(Dense(1, input_dim=2,activation=None))
model.compile(optimizer=Adam(0.001),
              loss="mean_squared_error",
              metrics=['mae'])


# Train model on dataset

# 注意： 这个generator 只能用1个workers，不能两个进程一起读文件
# 如果要使用多头读文件，需要更改__data_generation  并准备多份文件
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5)

Epoch 1/5
3125/3125 [==============================] - 5s 1ms/step - loss: 0.8113 - mean_absolute_error: 0.6224 - val_loss: 0.0470 - val_mean_absolute_error: 0.1758
Epoch 2/5
3125/3125 [==============================] - 4s 1ms/step - loss: 0.0181 - mean_absolute_error: 0.1020 - val_loss: 0.0055 - val_mean_absolute_error: 0.0600
Epoch 3/5
3125/3125 [==============================] - 4s 1ms/step - loss: 0.0052 - mean_absolute_error: 0.0592 - val_loss: 0.0051 - val_mean_absolute_error: 0.0582
Epoch 4/5
3125/3125 [==============================] - 4s 1ms/step - loss: 0.0051 - mean_absolute_error: 0.0591 - val_loss: 0.0051 - val_mean_absolute_error: 0.0581
Epoch 5/5
3125/3125 [==============================] - 4s 1ms/step - loss: 0.0051 - mean_absolute_error: 0.0591 - val_loss: 0.0051 - val_mean_absolute_error: 0.0581


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.weights

[<tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

In [19]:
W = model.layers[0]

## 查看weights

In [20]:
W.get_weights()

[array([[0.8518454],
        [1.6893313]], dtype=float32), array([0.9499666], dtype=float32)]

## 使用神经网络

多加隐藏层

In [21]:


# Generators
training_generator = DataGenerator(open("data.csv", 'r'), total_num_data=100000, num_features=2)
validation_generator = DataGenerator(open("data_test.csv", 'r'), total_num_data=100000, num_features=2)

# Design model

# 如果要改模型， 只需要改这个东西

model = Sequential()
model.add(Dense(20, input_dim=2,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation=None))
model.compile(optimizer=Adam(0.001),
              loss="mean_squared_error",
              metrics=['mae'])


# Train model on dataset

# 注意： 这个generator 只能用1个workers，不能两个进程一起读文件
# 如果要使用多头读文件，需要更改__data_generation  并准备多份文件
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5)

Epoch 1/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0472 - mean_absolute_error: 0.0650 - val_loss: 0.0012 - val_mean_absolute_error: 0.0291
Epoch 2/5
3125/3125 [==============================] - 5s 2ms/step - loss: 0.0010 - mean_absolute_error: 0.0267 - val_loss: 0.0014 - val_mean_absolute_error: 0.0305
Epoch 3/5
3125/3125 [==============================] - 5s 2ms/step - loss: 9.8576e-04 - mean_absolute_error: 0.0266 - val_loss: 0.0014 - val_mean_absolute_error: 0.0310
Epoch 4/5
3125/3125 [==============================] - 5s 2ms/step - loss: 9.7953e-04 - mean_absolute_error: 0.0265 - val_loss: 0.0015 - val_mean_absolute_error: 0.0315
Epoch 5/5
3125/3125 [==============================] - 5s 2ms/step - loss: 9.7210e-04 - mean_absolute_error: 0.0264 - val_loss: 0.0015 - val_mean_absolute_error: 0.0312


##  怎么加变量

    * y1 = f1(x1, x2)
    * y2 = f1(y1, x3)
    * output y2

In [88]:
from keras.layers import Concatenate
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.optimizers import Adagrad

first_input = Input(shape=(2, ))
first_dense = Dense(1, )(first_input)

second_input = Input(shape=(1, ))
second_dense = Dense(1, )(second_input)

merge = concatenate([first_dense, second_dense])
output = Dense(1,)(merge)

model = Model(inputs=[first_input, second_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error',
               metrics=['mae'])


In [89]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 1)            3           input_9[0][0]                    
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 1)            2           input_10[0][0]                   
__________________________________________________________________________________________________
concatenat

模型中需要两个input，所以需要更改data generator， 每次data_generation 一个列表

In [90]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, file_cursor, total_num_data, num_features, batch_size=32,shuffle=True):
        """
            file_cursor is an opened file, WITHOUT .read() 
        """
        self.total_num_data = total_num_data
        self.num_features = num_features
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_cursor = file_cursor
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.total_num_data / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'

        X, y = self.__data_generation()

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch, reopen the file'
        self.file_cursor.seek(0)

    def __data_generation(self):
        """read self.batch_size of examples"""
        
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.num_features))
        X2 = np.empty((self.batch_size, 1))
        y = np.empty((self.batch_size,))

        for i in range(self.batch_size):
            line = self.file_cursor.readline()
            # parse string to numpy
            line = [float(v) for v in line[:-1].split(',')]
            X[i,:] = line[:-1]
            X2[i] = line[0]
            y[i] = line[-1]
        
        return [X, X2], y

In [91]:
training_generator = DataGenerator(open("data.csv", 'r'), total_num_data=100000, num_features=2)
validation_generator = DataGenerator(open("data_test.csv", 'r'), total_num_data=100000, num_features=2)

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5)

Epoch 1/5
3125/3125 [==============================] - 7s 2ms/step - loss: 0.6993 - mean_absolute_error: 0.5563 - val_loss: 0.0944 - val_mean_absolute_error: 0.2612
Epoch 2/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0928 - mean_absolute_error: 0.2596 - val_loss: 0.0934 - val_mean_absolute_error: 0.2602
Epoch 3/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0928 - mean_absolute_error: 0.2596 - val_loss: 0.0934 - val_mean_absolute_error: 0.2602
Epoch 4/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0928 - mean_absolute_error: 0.2596 - val_loss: 0.0934 - val_mean_absolute_error: 0.2602
Epoch 5/5
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0928 - mean_absolute_error: 0.2596 - val_loss: 0.0934 - val_mean_absolute_error: 0.2602
